In [ ]:
import requests
import re
import google.generativeai as genai
genai.configure(api_key="")

model = genai.GenerativeModel('gemini-1.5-flash')

# API Keys
ETHERSCAN_API_KEY = ''
MORALIS_API_KEY = ''

# Function to identify and parse the Ethereum address or transaction hash from the prompt
def parse_prompt(prompt):
    address = re.search(r'0x[a-fA-F0-9]{40}', prompt)
    tx_hash = re.search(r'0x[a-fA-F0-9]{64}', prompt)
    
    address = address.group() if address else None
    tx_hash = tx_hash.group() if tx_hash else None
    
    return address, tx_hash, prompt

# Function to fetch transaction details from Etherscan
def fetch_tx_details(tx_hash):
    url = f'https://api.etherscan.io/api?module=proxy&action=eth_getTransactionByHash&txhash={tx_hash}&apikey={ETHERSCAN_API_KEY}'
    response = requests.get(url)
    return response.json()

# Function to fetch wallet holdings from Moralis
def fetch_wallet_holdings(address):
    url = f'https://deep-index.moralis.io/api/v2.2/wallets/{address}/net-worth'
    headers = {
        'accept': 'application/json',
        'X-API-Key': MORALIS_API_KEY
    }
    response = requests.get(url, headers=headers)
    return response.json()

# Function to fetch ETH balance from Moralis
def fetch_eth_balance(address):
    url = f'https://deep-index.moralis.io/api/v2/{address}/balance?chain=eth'
    headers = {
        'accept': 'application/json',
        'X-API-Key': MORALIS_API_KEY
    }
    response = requests.get(url, headers=headers)
    return response.json()


# Function to generate response using OpenAI's GPT-3
def generate_response(prompt, data):

    response = model.generate_content(f"Given the prompt: '{prompt}' and the data: '{data}', generate a response.")

    return response.text

# Main function to process the prompt and fetch data
def process_prompt(prompt):
    address, tx_hash, action_prompt = parse_prompt(prompt)
    
    if tx_hash:
        # Handle transaction hash scenarios
        tx_details = fetch_tx_details(tx_hash)
        if 'sender' in action_prompt or 'receiver' in action_prompt:
            wallet_address = tx_details['result']['from'] if 'sender' in action_prompt else tx_details['result']['to']
            holdings = fetch_wallet_holdings(wallet_address)
            eth_balance = fetch_eth_balance(wallet_address)['balance']
            data = {'transaction_details': tx_details, 'holdings': holdings, 'eth_balance': eth_balance}
        else:
            data = tx_details
    else:
        return "No valid Ethereum address or transaction hash found in the prompt."
    
    response = generate_response(action_prompt, data)
    return response

# Example usage
prompt = "Fetch the sender of the transaction 0xae25a79e82cb2b501a5b60c7f1ab6f4989a1b408ecf640230a86515736a8f50f also tell me detail about sender wallet holdings"
result = process_prompt(prompt)
print(result)